In [43]:
import numpy as np
import pandas as pd
import os
import tempfile

import matplotlib.pyplot as plt

%matplotlib inline

In [44]:
from os import listdir
from os.path import isfile, join
path_train = 'ignored_assets/public_data/training_data/order_data'
trainfiles = [f for f in listdir(path_train) if isfile(join(path_train, f)) and f != '.DS_Store']

path_test1 = 'ignored_assets/public_data/test_set_1/order_data'
testfiles1 = [f for f in listdir(path_test1) if isfile(join(path_test1, f)) and f != '.DS_Store']

path_test2 = 'ignored_assets/public_data/test_set_2/order_data'
testfiles2 = [f for f in listdir(path_test2) if isfile(join(path_test2, f)) and f != '.DS_Store']


In [45]:
orderdf = []
for f in trainfiles:
    orderdf.append(pd.read_csv(join(path_train, f), delimiter='\t', header = None))
    
for f in testfiles1:
    orderdf.append(pd.read_csv(join(path_test1, f), delimiter='\t', header = None))
    
for f in testfiles2:
    orderdf.append(pd.read_csv(join(path_test2, f), delimiter='\t', header = None))




In [46]:
orderdf = pd.concat(orderdf)

In [47]:
#Clean up
orderdf[6] = pd.to_datetime(orderdf[6])
orderdf.columns = ['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash', 'price', 'timestamp']


In [48]:
# Create map of driver IDs
driver_ids=orderdf['driver_id'].unique()

In [49]:
#Index stuff
myindex=pd.MultiIndex.from_arrays([orderdf['driver_id'],orderdf['order_id']],names=('driver_id','order_id'))
orderdf2=orderdf.set_index(myindex)
orderdf2.sort_index()

%time orderdf2.loc[driver_ids[0]]

CPU times: user 82.1 ms, sys: 35.5 ms, total: 118 ms
Wall time: 118 ms


,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,timestamp
order_id,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23
5e03c747494657155b21320fa49144a9,5e03c747494657155b21320fa49144a9,dd65fa250fca2833a3a8c16d2cf0457c,da8ccb4ff0f510e46bb4643b8a785242,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,26.0,2016-01-01 10:30:13
236c00d175a2295ade03ab6ea4b9689a,236c00d175a2295ade03ab6ea4b9689a,dd65fa250fca2833a3a8c16d2cf0457c,ce27cf85aaa48764f650414d71f3674e,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:22:58
ea13ec3fe3b639eaefc36e4b54ebfd8e,ea13ec3fe3b639eaefc36e4b54ebfd8e,dd65fa250fca2833a3a8c16d2cf0457c,729164967a31e6befeecc44548220cab,d4ec2125aff74eded207d2d915ef682f,82a0ceee5481d552b2bab973b5ed85cb,22.0,2016-01-01 10:31:14
a741062b69f6f1828d69a3f771aab6d5,a741062b69f6f1828d69a3f771aab6d5,dd65fa250fca2833a3a8c16d2cf0457c,1dc3e244fdef29a3299722f9aa26720f,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:35:26
219e2dc27791817db73e877d0451ac7b,219e2dc27791817db73e877d0451ac7b,dd65fa250fca2833a3a8c16d2cf0457c,3084402739dc8b47e820c24b8ab06195,3e12208dd0be281c92a6ab57d9a6fb32,82a0ceee5481d552b2bab973b5ed85cb,20.4,2016-01-01 14:10:12
c88282a71e988c5fb7eb11e33d5ca512,c88282a71e988c5fb7eb11e33d5ca512,dd65fa250fca2833a3a8c16d2cf0457c,adbd59353c4651fc77405876d9e92638,62afaf3288e236b389af9cfdc5206415,d4ec2125aff74eded207d2d915ef682f,11.0,2016-01-01 09:43:39
a174cc46d34b2b73bbb9b94a6d022b80,a174cc46d34b2b73bbb9b94a6d022b80,dd65fa250fca2833a3a8c16d2cf0457c,5d8886092bbb717e0ccf37cb2d7a9661,d4ec2125aff74eded207d2d915ef682f,d4ec2125aff74eded207d2d915ef682f,9.7,2016-01-01 10:04:50
80559050428f704e024376a9d7306db9,80559050428f704e024376a9d7306db9,dd65fa250fca2833a3a8c16d2cf0457c,4c5946a01d3764886040ba18fac717d6,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,11.0,2016-01-01 10:28:18


In [50]:
def count_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf2.loc[driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    if len(driver_orders)>0:
        return float(sum(driver_orders['emptytrip'] & ~driver_orders['break']))/float(len(driver_orders))
    else:
        return np.NaN

In [53]:
count_empty_trips(driver_ids[31])

0.30726256983240224

In [54]:
driver_pd=pd.DataFrame(driver_ids[~pd.isnull(driver_ids)],columns=['driver_id'])

In [ ]:
driver_pd['frac_reb']=driver_pd.apply(lambda x: count_empty_trips(x['driver_id']),axis=1)

In [ ]:
orderdf2=orderdf.sort_values('driver_id')

myindex=pd.MultiIndex.from_tuples(list(zip(orderdf['driver_id'],orderdf['order_id'])))

orderdf3=orderdf2.set_index('driver_id')


#driver_orders=orderdf[orderdf['driver_id']==driver_hash].copy()

In [40]:
driver_ids[pd.isnull(driver_ids)]

array([nan], dtype=object)

CPU times: user 40.2 ms, sys: 28.9 ms, total: 69.1 ms
Wall time: 67.8 ms


,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,timestamp
order_id,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23
5e03c747494657155b21320fa49144a9,5e03c747494657155b21320fa49144a9,dd65fa250fca2833a3a8c16d2cf0457c,da8ccb4ff0f510e46bb4643b8a785242,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,26.0,2016-01-01 10:30:13
236c00d175a2295ade03ab6ea4b9689a,236c00d175a2295ade03ab6ea4b9689a,dd65fa250fca2833a3a8c16d2cf0457c,ce27cf85aaa48764f650414d71f3674e,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:22:58
ea13ec3fe3b639eaefc36e4b54ebfd8e,ea13ec3fe3b639eaefc36e4b54ebfd8e,dd65fa250fca2833a3a8c16d2cf0457c,729164967a31e6befeecc44548220cab,d4ec2125aff74eded207d2d915ef682f,82a0ceee5481d552b2bab973b5ed85cb,22.0,2016-01-01 10:31:14
a741062b69f6f1828d69a3f771aab6d5,a741062b69f6f1828d69a3f771aab6d5,dd65fa250fca2833a3a8c16d2cf0457c,1dc3e244fdef29a3299722f9aa26720f,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:35:26
219e2dc27791817db73e877d0451ac7b,219e2dc27791817db73e877d0451ac7b,dd65fa250fca2833a3a8c16d2cf0457c,3084402739dc8b47e820c24b8ab06195,3e12208dd0be281c92a6ab57d9a6fb32,82a0ceee5481d552b2bab973b5ed85cb,20.4,2016-01-01 14:10:12
c88282a71e988c5fb7eb11e33d5ca512,c88282a71e988c5fb7eb11e33d5ca512,dd65fa250fca2833a3a8c16d2cf0457c,adbd59353c4651fc77405876d9e92638,62afaf3288e236b389af9cfdc5206415,d4ec2125aff74eded207d2d915ef682f,11.0,2016-01-01 09:43:39
a174cc46d34b2b73bbb9b94a6d022b80,a174cc46d34b2b73bbb9b94a6d022b80,dd65fa250fca2833a3a8c16d2cf0457c,5d8886092bbb717e0ccf37cb2d7a9661,d4ec2125aff74eded207d2d915ef682f,d4ec2125aff74eded207d2d915ef682f,9.7,2016-01-01 10:04:50
80559050428f704e024376a9d7306db9,80559050428f704e024376a9d7306db9,dd65fa250fca2833a3a8c16d2cf0457c,4c5946a01d3764886040ba18fac717d6,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,11.0,2016-01-01 10:28:18


In [ ]:
list(zip(orderdf['driver_id'],orderdf['order_id']))

In [ ]:
count_empty_trips(driver_ids[0])

In [ ]:
#Create a timeline of rebalancing trips.
# For each, record two timestamps (last pickup, next pickup), start location, and end location.
# Sketch: for each driver, return a list of reb trips by masking. Then append.
# Can we do this in a non-sequential way?

In [ ]:
def find_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf2.loc[driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    return driver_orders[driver_orders['emptytrip'] & ~driver_orders['break']]

In [ ]:
empty_rides_list=driver_pd.apply(lambda x: find_empty_trips(x['driver_id']),axis=1)